# Biography Generator - Interactive Notebook

This notebook generates professional narratives, obituaries, or dating app introductions from uploaded documents using Claude AI.

**Features:**
- Load PDF or TXT documents
- Generate three types of biographies
- Iterative refinement with conversation history
- Export conversation history showing how outputs become inputs

In [ ]:
import anthropic
from pypdf import PdfReader
import os
from datetime import datetime
import glob
from IPython.display import display, Markdown
## 2. Helper Functions

Functions to extract text from different file formats.
def extract_text_from_pdf(file_path):
    """Extract text from a PDF file."""
    try:
        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return None


def read_text_file(file_path):
    """Read text from a text file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f"Error reading text file: {e}")
        return None
## 3. BiographyGenerator Class

Main class that manages document loading and biography generation with conversation history.

**Key Concept:** The conversation history list stores all messages (user and assistant) and is sent with each API call. This allows Claude to remember context and refine its previous outputs.
class BiographyGenerator:
    """Manages document loading and biography generation with conversation history."""
    
    def __init__(self, api_key):
        self.client = anthropic.Anthropic(api_key=api_key)
        self.conversation_history = []  # Stores all messages for context
        self.document_content = None
        
    def load_document(self, file_path):
        """Load document content from a file."""
        if not os.path.exists(file_path):
            print(f"❌ File not found: {file_path}")
            return False
            
        # Determine file type
        if file_path.lower().endswith('.pdf'):
            self.document_content = extract_text_from_pdf(file_path)
        elif file_path.lower().endswith(('.txt', '.text')):
            self.document_content = read_text_file(file_path)
        else:
            print("❌ Unsupported file type. Please use .pdf or .txt files.")
            return False
            
        if self.document_content:
            print(f"✅ Document loaded successfully!")
            print(f"   Characters: {len(self.document_content)}")
            print(f"   Words: ~{len(self.document_content.split())}")
            return True
        return False
    
    def generate_content(self, output_type):
        """
        Generate content based on the selected type.
        
        Args:
            output_type: 1 (Professional narrative), 2 (Obituary), or 3 (Dating app intro)
        """
        if not self.document_content:
            return "❌ Please load a document first!"
        
        # Define prompts for each type
        prompts = {
            1: """Based on the following information about a person, please write a professional narrative biography. 
This should be suitable for a professional website, LinkedIn profile, or corporate bio. 
Focus on career achievements, expertise, education, and professional impact.
Make it compelling but formal and professional.

Document content:
{content}

Generate a well-structured professional narrative (approximately 150-300 words).""",
            
            2: """Based on the following information about a person, please write a thoughtful and respectful obituary.
Include key life events, achievements, family connections, and what made them special.
Strike a balance between celebrating their life and acknowledging the loss.

Document content:
{content}

Generate a well-structured obituary (approximately 200-400 words).""",
            
            3: """Based on the following information about a person, please write an engaging dating app introduction.
Make it authentic, warm, and interesting. Highlight personality, interests, values, and what makes them unique.
Use a conversational, friendly tone. Be genuine and not overly formal.
Include what they're looking for in a partner if that information is available.

Document content:
{content}

Generate a compelling dating app bio (approximately 100-200 words)."""
        }
        
        prompt = prompts[output_type].format(content=self.document_content)
        
        # Start new conversation
        self.conversation_history = [
            {"role": "user", "content": prompt}
        ]
        
        # Call Claude API
        print("🤔 Generating content...\n")
        response = self.client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=2000,
            messages=self.conversation_history
        )
        
        # Add Claude's response to history
        assistant_message = response.content[0].text
        self.conversation_history.append({
            "role": "assistant",
            "content": assistant_message
        })
        
        return assistant_message
    
    def follow_up(self, user_message):
        """Send a follow-up message to refine the content."""
        if not self.conversation_history:
            return "❌ Please generate content first!"
        
        # Add user's follow-up to conversation
        self.conversation_history.append({
            "role": "user",
            "content": user_message
        })
        
        # Call Claude with full conversation history
        print("🤔 Processing your request...\n")
        response = self.client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=2000,
            messages=self.conversation_history
        )
        
        # Add Claude's response to history
        assistant_message = response.content[0].text
        self.conversation_history.append({
            "role": "assistant",
            "content": assistant_message
        })
        
        return assistant_message
    
    def export_conversation(self, filename=None):
        """
        Export the full conversation history to a text file.
        This shows how outputs from Claude are reused as inputs in subsequent turns.
        """
        if not self.conversation_history:
            print("❌ No conversation to export!")
            return None
        
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"conversation_history_{timestamp}.txt"
        
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                f.write("="*80 + "\n")
                f.write("BIOGRAPHY GENERATOR - CONVERSATION HISTORY\n")
                f.write("="*80 + "\n")
                f.write(f"Exported: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write(f"Total Messages: {len(self.conversation_history)}\n")
                f.write("="*80 + "\n\n")
                
                f.write("NOTE: This conversation demonstrates how Claude's outputs are reused as\n")
                f.write("inputs in subsequent API calls by maintaining conversation history.\n")
                f.write("Each message includes the full context of all previous messages.\n\n")
                f.write("="*80 + "\n\n")
                
                for i, message in enumerate(self.conversation_history, 1):
                    role = "USER" if message["role"] == "user" else "CLAUDE"
                    content = message["content"]
                    
                    f.write(f"\n{'='*80}\n")
                    f.write(f"MESSAGE {i}: {role}\n")
                    f.write(f"{'='*80}\n\n")
                    f.write(content)
                    f.write("\n\n")
                    
                    # Add explanation after first exchange
                    if i == 2:
                        f.write(f"\n{'~'*80}\n")
                        f.write("📝 CONVERSATION CONTEXT NOTE:\n")
                        f.write("From this point forward, every API call includes BOTH messages above.\n")
                        f.write("Claude can reference its previous response and the original document.\n")
                        f.write(f"{'~'*80}\n\n")
                
                # Add summary at the end
                f.write(f"\n{'='*80}\n")
                f.write("CONVERSATION SUMMARY\n")
                f.write(f"{'='*80}\n\n")
                f.write(f"Total exchanges: {len(self.conversation_history) // 2}\n")
                f.write(f"User messages: {sum(1 for m in self.conversation_history if m['role'] == 'user')}\n")
                f.write(f"Claude responses: {sum(1 for m in self.conversation_history if m['role'] == 'assistant')}\n\n")
                f.write("This transcript demonstrates:\n")
                f.write("1. How initial prompts establish context\n")
                f.write("2. How Claude's outputs inform subsequent inputs\n")
                f.write("3. How conversation history enables iterative refinement\n")
                f.write("4. Proper conversation management for LLM applications\n")
            
            return filename
            
        except Exception as e:
            print(f"❌ Error exporting conversation: {e}")
            return None
## 4. Setup - API Key

#Set API key directly (not recommended for shared notebooks)
# API_KEY = "your-api-key-here"

if not API_KEY:
    API_KEY = input("Enter your Anthropic API key: ").strip()

# Initialize the generator
generator = BiographyGenerator(API_KEY)
print("✅ Generator initialized!")
## 5. Load Document

Browse available files or enter a file path manually.
# Find available PDF and TXT files
pdf_files = glob.glob("*.pdf") + glob.glob("**/*.pdf", recursive=False)
txt_files = glob.glob("*.txt") + glob.glob("**/*.txt", recursive=False)
all_files = sorted(set(pdf_files + txt_files))

print("📁 Available files:\n")
for i, file in enumerate(all_files, 1):
    file_size = os.path.getsize(file) / 1024
    file_type = "PDF" if file.endswith('.pdf') else "TXT"
    print(f"  {i}. {file:<50} ({file_size:.1f} KB) [{file_type}]")

if not all_files:
    print("❌ No files found. Please add PDF or TXT files to the directory.")
# Select a file by number or enter path manually
# Option 1: Select by number
file_number = 1  # Change this to select different file

# Option 2: Or enter path directly
# file_path = "path/to/your/file.pdf"

if all_files:
    file_path = all_files[file_number - 1]
    print(f"Selected: {file_path}\n")
    generator.load_document(file_path)
else:
    print("Please add a file and run this cell again")
# Select output type (1, 2, or 3)
output_type = 1  # Change this to 1, 2, or 3

type_names = {
    1: "Professional Narrative",
    2: "Obituary",
    3: "Dating App Introduction"
}

result = generator.generate_content(output_type)

# Display result with formatting
display(Markdown(f"## {type_names[output_type]}"))
display(Markdown(result))
# Enter your refinement request
refinement = "Make it shorter and more engaging"  # Customize this

result = generator.follow_up(refinement)

# Display refined result
display(Markdown(f"## Refined {type_names[output_type]}"))
display(Markdown(result))
# Save current content
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"{type_names[output_type].lower().replace(' ', '_')}_{timestamp}.txt"

with open(output_file, 'w', encoding='utf-8') as f:
    f.write(f"{type_names[output_type].upper()}\n")
    f.write("="*70 + "\n\n")
    f.write(result)
    f.write("\n\n" + "="*70 + "\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

print(f"✅ Content saved to: {output_file}")
# Export conversation history
filename = generator.export_conversation()

if filename:
    print(f"✅ Conversation history exported to: {filename}")
    print("\nThis file shows:")
    print("  • How initial prompts establish context")
    print("  • How Claude's outputs inform subsequent inputs")
    print("  • How conversation history enables iterative refinement")
# Display conversation history
print(f"Total messages in conversation: {len(generator.conversation_history)}\n")

for i, message in enumerate(generator.conversation_history, 1):
    role = message['role'].upper()
    content = message['content']
    
    # Truncate long messages for display
    if len(content) > 500:
        content_preview = content[:500] + "\n... [truncated]"
    else:
        content_preview = content
    
    print(f"\n{'='*70}")
    print(f"MESSAGE {i}: {role}")
    print(f"{'='*70}")
    print(content_preview)
    print()